<a href="https://colab.research.google.com/github/rahulkg666/lmsaibotcolab/blob/master/LMSAIBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U langchain langchain-openai langchain_community python-dotenv langchain-chroma newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setu

In [5]:
%env OPENAI_API_KEY=sk-proj-KrtXWRSyglCMUfVD1rBjT3BlbkFJrZ1d8ju1Ib5xvFVGG6dQ
%env USER_AGENT=rahul.bhaskar@perpetualtp.com

env: OPENAI_API_KEY=sk-proj-KrtXWRSyglCMUfVD1rBjT3BlbkFJrZ1d8ju1Ib5xvFVGG6dQ
env: USER_AGENT=rahul.bhaskar@perpetualtp.com


In [6]:
LANGCHAIN_TRACING_V2=True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="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"

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [37]:
import os
import openai
import sys
import datetime
sys.path.append('../..')
from langchain.document_loaders import WebBaseLoader, NewsURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.utils import filter_complex_metadata
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
def currentLLM():
  current_date = datetime.datetime.now().date()
  llm_name=''
  if current_date < datetime.date(2023, 9, 2):
      llm_name = "gpt-3.5-turbo-0301"
  else:
      llm_name = "gpt-3.5-turbo"
  return llm_name
def getContent(websiteURL):
  #["https://slxlearning.com/contact-us","https://slxlearning.com/about-us","https://slxlearning.com","https://slxlearning.com/sustainability-readiness-programme/","https://slxlearning.com/programs-sustainability-coach/","https://slxlearning.com/programs-certified-sustainability-professional-cspr/"]
  loader = NewsURLLoader(websiteURL)
  data = loader.load()
  return data
def splitData(data):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 0
  )
  splits = text_splitter.split_documents(data)
  #len(splits)
  return splits
def promptTemplate():
  template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
  {context}
  Question: {question}
  Helpful Answer:"""
  QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)
  return QA_CHAIN_PROMPT
def chatBot(question, chain_type, k):
  llm = ChatOpenAI(model_name=currentLLM(), temperature=0)
  websiteUrls = ["https://slxlearning.com/contact-us","https://slxlearning.com/about-us","https://slxlearning.com","https://slxlearning.com/sustainability-readiness-programme/","https://slxlearning.com/programs-sustainability-coach/","https://slxlearning.com/programs-certified-sustainability-professional-cspr/"]
  data = getContent(websiteUrls)
  splits = splitData(data)
  persist_directory = '/content/docs/chroma/'
  embedding = OpenAIEmbeddings()
  #vectordb = Chroma.from_documents(documents = splits,persist_directory=persist_directory, embedding_function=embedding)
  vectordb = Chroma.from_documents(
      documents = filter_complex_metadata(splits),
      embedding = embedding,
      persist_directory = persist_directory
  )
  #print(vectordb._collection.count())
  retriever = vectordb.as_retriever(search_type="mmr") #as_retriever(search_type="similarity", search_kwargs={"k": k})
  memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
  )
  qa_chain = ConversationalRetrievalChain.from_llm(llm,
        chain_type=chain_type,
        retriever=retriever,
        #return_source_documents=True,
        #chain_type_kwargs={"prompt": promptTemplate()},
        #return_generated_question=True,
        memory=memory
    )
  result = qa_chain.invoke(question)
  #print(result)
  return result['answer']

#response = chatBot("What are the modules in CASM?")
#print(response)




In [36]:
response = chatBot("What are the modules in CASM?","refine",2)
print(response)

The modules in CASM cover a range of topics to provide learners with a comprehensive understanding of sustainable finance and its various aspects. These modules include:

1. Introduction to Sustainable Finance
2. Sustainable Financial Modelling
3. Responsible Investment
4. Sustainable Finance Products
5. Economics of transitioning to a lower carbon future
6. Financing green assets
7. Emissions trading schemes
8. Green bonds
9. Role of stock exchanges in sustainable finance
10. International developments in UNFCCC

Additionally, the course aims to make learners aware of the evolving view of financial activities and the inclusion of sustainability in financial decision-making. It introduces emerging tools in sustainable finance and equips learners to understand and work in fields related to pricing carbon, conduct risk assessments related to climate change, manage climate risk disclosures, handle sustainable asset management, and understand the role of insurance in climate risk mitigatio